In [1]:
import pandas as pd
import numpy as np


In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("serie-a-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("serie-a-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("serie-a-2025.csv")

## Obavezno!!!

In [3]:
#odrediti duzinu sezone
df_25_6 = df_25_6.head(120)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
115,113,12,23/11/2025 14:00,Giovanni Zini,Cremonese,Roma,1 - 3
116,116,12,23/11/2025 17:00,Olimpico,Lazio,Lecce,2 - 0
117,115,12,23/11/2025 19:45,Giuseppe Meazza,Inter,Milan,0 - 1
118,119,12,24/11/2025 17:30,Olimpico Grande Torino,Torino,Como,1 - 5
119,118,12,24/11/2025 19:45,Mapei Stadium - Città del Tricolore,Sassuolo,Pisa,2 - 2


## Priprema Podataka i df-a

In [4]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

rez_dom = df_BL["Result"].str.slice(0,1)
rez_gost = df_BL["Result"].str.slice(4,5)

df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]

df1 = df_rezultatski

## Definisanje Funckije za Izracun Forme

In [5]:
def klub(tim):
    # Filtriramo DataFrame
    df = df1[(df1["H"] == tim) | (df1["A"] == tim)].copy()
    
    uslovi = [
    (df['H'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] < df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] == df["Rez2"]),
    (df['H'] == tim) & (df['Rez1'] > df["Rez2"]),
    (df['A'] == tim) & (df['Rez1'] > df["Rez2"]),]
    
    vrijednosti = [-1, 1, 0, 0, 1, -1]
    df['Forma'] = np.select(uslovi, vrijednosti, default=0)
    df = df.drop(["Rez1","Rez2"],axis=1).tail(5)
    rezultat = df["Forma"].sum() / len(df["Forma"])
    return rezultat

## Provjera Rada te Funckije

In [6]:
klub('Inter')

0.2

## Stvaranje Petlje za Automatski Izracun Forme Svih Klubova

In [7]:
#izvlacenje naziva svih klubova
timovi = pd.concat([df1["H"], df1["A"]]).unique()
timovi

array(['Empoli', 'Frosinone', 'Genoa', 'Inter', 'Roma', 'Sassuolo',
       'Lecce', 'Udinese', 'Torino', 'Bologna', 'Monza', 'Milan',
       'Hellas Verona', 'Fiorentina', 'Juventus', 'Lazio', 'Napoli',
       'Salernitana', 'Cagliari', 'Atalanta', 'Parma', 'Venezia', 'Como',
       'Cremonese', 'Pisa'], dtype=object)

In [8]:
rezultati = {}

In [9]:
#petlja
for tim in timovi:
    rezultat = klub(tim)
    rezultati[tim] = rezultat

# Kreiranje DataFrame-a iz rezultata
df_forme = pd.DataFrame(list(rezultati.items()),columns = ["Klub","Rezultat"])
df_forme["Rezultat"] = df_forme["Rezultat"]

## Rezulat Forme za Sve

In [10]:
#Izracun
df_forme 

,Klub,Rezultat
0,Empoli,-0.2
1,Frosinone,0.0
2,Genoa,-0.2
3,Inter,0.2
4,Roma,0.6
5,Sassuolo,0.0
6,Lecce,-0.4
7,Udinese,-0.2
8,Torino,0.0
9,Bologna,0.6


In [11]:
#Ponovna provjera vrijednosti...
klub("Inter")

0.2

In [12]:
df_forme.to_csv("forma_h.csv")

In [13]:
df_forme["Rezultat"] = df_forme["Rezultat"] * -1

In [14]:
df_forme


,Klub,Rezultat
0,Empoli,0.2
1,Frosinone,-0.0
2,Genoa,0.2
3,Inter,-0.2
4,Roma,-0.6
5,Sassuolo,-0.0
6,Lecce,0.4
7,Udinese,0.2
8,Torino,-0.0
9,Bologna,-0.6


In [15]:
df_forme.to_csv("forma_a.csv")